In [2]:
# import necessary dependencies
from itertools import product as itp
from pythtb import *
from triqs.lattice.tight_binding import TBLattice
import sympy as sp
import warnings
import numpy as np
import matplotlib.pyplot as plt

Starting serial run at: 2023-08-04 10:05:35.762577


In [3]:
from triqs.lattice.utils import TB_from_pythTB
w90_input = w90('La2CuO4 Files', 'lco')
fermi_ev = 12.7367
w90_model = w90_input.model(zero_energy = fermi_ev, min_hopping_norm = 0.038, max_distance = None)
w90_triqs = TB_from_pythTB(w90_model)

In [14]:
def compare_minimal_models(w90_input, fermi_ev, cutoff_1 = 0.038, cutoff_2 = 0.05, cutoff_3 = 0.45):
    r"""
    returns a figure that compares 3 minimal tight-binding models against the full Wannier90 model
    
    Parameters
    ----------
    w90_input: pythtb.w90
        a reading of the output files from Wannier90
    fermi_ev: float
        the energy, in eV, at the fermi level. Can be gotten from the density-functional code
    cutoff_1: float, default = 0.038
        the first hopping norm/amplitude cutoff in eV
    cutoff_2: float, default = 0.05
        the second hopping norm/amplitude cutoff in eV
    cutoff_3: float, default = 0.45
        the third hopping norm/amplitude cutoff in eV

    """

    # get band structure from Wannier90 in the form of
    # list of k-points in reduced coordinates
    # energies interpolated by Wannier90
    (w90_kpt, w90_evals) = w90_input.w90_bands_consistency()

    # obtain simplified models based on various cutoffs
    w90_model_1 = w90_input.model(zero_energy = fermi_ev, min_hopping_norm = cutoff_1, max_distance = None)
    w90_model_2 = w90_input.model(zero_energy = fermi_ev, min_hopping_norm = cutoff_2, max_distance = None)
    w90_model_3 = w90_input.model(zero_energy = fermi_ev, min_hopping_norm = cutoff_3, max_distance = None)

    # solve simplified models on the same k-path as in Wannier90
    int_evals_1 = w90_model_1.solve_all(w90_kpt)
    int_evals_2 = w90_model_2.solve_all(w90_kpt)
    int_evals_3 = w90_model_3.solve_all(w90_kpt)

    # setting up the plots
    fig, ax = plt.subplots(dpi = 2000)

    # plotting the full Wannier90 model
    for i in range(w90_evals.shape[0]):
        x_axis = list(range(w90_evals.shape[1]))
        y_axis = w90_evals[i] - fermi_ev
        ax.plot(x_axis, y_axis, "k-", zorder = -100)

    # plotting the minimal models
    for i in range(int_evals_1.shape[0]):
        ax.plot(list(range(int_evals_1.shape[1])), int_evals_1[i], "r-", zorder = -50)
        ax.plot(list(range(int_evals_2.shape[1])), int_evals_2[i], "g-", zorder = -50)
        ax.plot(list(range(int_evals_3.shape[1])), int_evals_3[i], "c-", zorder = -50)
    
    # plotting the horizontal line that passes through the y-axis
    ax.axhline(y = 0, color = 'm', linestyle = '--')
    ax.set_yticks(np.arange(-2, 2.1, step = 1))

    # increasing the width of the curves
    lines = ax.lines
    for line in lines:
        line.set_linewidth(2.5)
    
    # coordinates of the high symmetry points
    G = np.array([0.00, 0.00, 0.00])
    N = np.array([0.25, -0.25, 0.25])
    X = np.array([0.00, 0.00, 0.50])
    M = np.array([0.25, 0.25, -0.25])

    # x-axis labels of the high-symmetry points
    kpath_labels = ['G', 'N', 'X', 'G', 'M']
    idx_kpt = {}

    # checking k-point coordinates equal to the high-symmetry point coordinates
    for name, idx in (zip(('G', 'N', 'X', 'M'), [G, N, X, M])):
        idx_kpt[name] = list(np.where((w90_kpt == idx).all(axis = 1))[0])
    
    # setting the x-axis ticks
    kpath_xticks = list([x[0] for x in idx_kpt.values()])
    kpath_xticks.append(idx_kpt['G'][1])
    kpath_xticks = sorted(kpath_xticks)
    ax.set_xticks(kpath_xticks)
    ax.set_xticklabels(kpath_labels)
    
    # setting the vertical lines at the high-symmetry points
    for n in range(len(kpath_xticks)):
        ax.axvline(x = kpath_xticks[n], linewidth = 1, color = 'k')

    # plot properties
    ax.set_xlim(0, int_evals_1.shape[1] - 1)
    ax.set_ylabel(r"Band Energy - $E_F$ (eV)")
    ax.set_title("Comparing Minimal Models")
    ax.legend(["Full model", "Cutoff at " + str(cutoff_1) + " eV", "Cutoff at " + str(cutoff_2) + " eV", "Cutoff at " + str(cutoff_3) + " eV"])


In [17]:
# calling the function
lanthanum_w90_input = w90('La2CuO4 Files', 'lco')
compare_minimal_models(lanthanum_w90_input, 12.7367)

<class 'pythtb.w90'>
<class 'float'>
